# 1 Visualize a Sample Image and its Segmentation Mask



In [ ]:
# Code to visualize a sample image and its segmentation mask

# Get data loaders
train_loader, val_loader, test_set = get_data_loaders(batch_size=16)

# Visualize a sample image and its segmentation mask
visualize_sample(train_loader)

# 2 Identify the Classes in the Code that is used to load the data

Q2.1: First of all identify the classes that are used to load the data.

A2.1: The classes are `DroneDataset` and `DroneTestDataset`.



Q2.2: What is the purpose of the `tiles` method in the `DroneDataset` class?

A2.2: The `tiles` method is used to create patches of the image and mask. This is done to handle large images that cannot be processed at once.



Q2.3: What is the purpose of the `DroneTestDataset` class? Why is it different from the `DroneDataset` class?

A2.3: The `DroneTestDataset` class is used to load the test data. It is different from the `DroneDataset` class because it does not perform any data augmentation.


Q2.4: What is the purpose of the `get_data_loaders` function?

A2.4: The `get_data_loaders` function is used to create the data loaders for the training, validation, and test sets.


# 3 Create the Model

Q3.1: Look at the `create_model` function. What is the first layer of the model?

A3.1: The first layer of the model is a `Conv2d` layer with 16 output channels, a kernel size of 3, and a stride of 1.


Q3.2: What is the purpose of the `Conv2d` layer?

A3.2: The `Conv2d` layer is used to extract features from the input image.

Q3.3: What is the purpose of the `MaxPool2d` layer?

A3.3: The `MaxPool2d` layer is used to reduce the spatial dimensions of the input image.

Q3.4: What is the purpose of the `Upsample` layer?

A3.4: The `Upsample` layer is used to increase the spatial dimensions of the input image.



# 4 Train the Model

Q4.1: What is the purpose of the `fit` function?

A4.1: The `fit` function is used to train the model.


Q4.2: What is the purpose of the `resume_from_checkpoint` function?

A4.2: The `resume_from_checkpoint` function is used to resume training from a checkpoint.





# 5 Inference

Q5.1: What is the purpose of the `evaluate_model` function?

A5.1: The `evaluate_model` function is used to evaluate the model on the test set.


# 6 Visualize Predictions

Q6.1: What is the purpose of the `visualize_predictions` function?

A6.1: The `visualize_predictions` function is used to visualize the predictions of the model on the test set.


# 7 Metrics

Q7.1: What is the purpose of the `mIoU` function?

A7.1: The `mIoU` function is used to calculate the mean Intersection over Union (mIoU) score.

Q7.2: What is the purpose of the `pixel_accuracy` function?

A7.2: The `pixel_accuracy` function is used to calculate the pixel accuracy score.


# 8 Inference - Intermediate Difficulty

Q8.1: Write the code to perform inference on a single image using the code provided in the notebook.

In [ ]:
def inference(model_path):
    # Get data loaders
    _, _, test_set = get_data_loaders(batch_size=16)

    # Load the trained model
    model = torch.load(model_path, map_location=device)
    print(f"Loaded model from {model_path}")

    # Evaluate on test set
    test_miou, test_accuracy = evaluate_model(model, test_set)
    print("Test Set mIoU:", test_miou)
    print("Test Set Pixel Accuracy:", test_accuracy)

    # Visualize predictions
    visualize_predictions(model, test_set, "test_predictions.pdf")


# 9 Train - Intermediate Difficulty

Q9.1: Write the code to train the model on the training set and save the model.


In [ ]:

def train_model():
    # Get data loaders
    train_loader, val_loader, test_set = get_data_loaders(batch_size=16)

    # Visualize a sample image and its segmentation mask
    visualize_sample(train_loader)

    # Create model
    model = create_model()

    # Training parameters
    max_lr = 1e-3
    epochs = 30
    weight_decay = 1e-4

    # Loss function
    criterion = nn.CrossEntropyLoss()

    # Optimizer
    optimizer = torch.optim.AdamW(
        model.parameters(), lr=max_lr, weight_decay=weight_decay
    )

    # Learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, max_lr, epochs=epochs, steps_per_epoch=len(train_loader)
    )

    # Check for existing checkpoints and resume training if possible
    checkpoint_dir = "checkpoints"
    os.makedirs(checkpoint_dir, exist_ok=True)
    model, optimizer, sched, start_epoch, min_loss, history = resume_from_checkpoint(
        model, optimizer, sched, checkpoint_dir
    )

    # Train the model
    history = fit(
        epochs - start_epoch,
        model,
        train_loader,
        val_loader,
        criterion,
        optimizer,
        sched,
        checkpoint_dir=checkpoint_dir,
    )

    # Save the final model
    torch.save(model, "Unet-Mobilenet.pt")

    # Plot training results
    plot_loss(history)
    plot_score(history)
    plot_acc(history)

    # Evaluate on test set
    test_miou, test_accuracy = evaluate_model(model, test_set)
    print("Test Set mIoU:", test_miou)
    print("Test Set Pixel Accuracy:", test_accuracy)

    # Visualize predictions
    visualize_predictions(model, test_set, "test_predictions.pdf")

    return model